In [1]:
!pip install transformers datasets scikit-learn pandas numpy sentence-transformers pyyaml


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [82]:
from transformers import  AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, DataCollatorForLanguageModeling
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset
import pandas as pd
import numpy as np
import warnings
import logging
import random
import torch
import json
import yaml
import os
import json
import torch
from transformers import pipeline
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [83]:
warnings.filterwarnings("ignore")

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)
system_prompt_file = config['arr_prompt']

with open(system_prompt_file,'r') as file:
    system_prompt = file.read()



In [84]:
print(system_prompt)

You are a Legal Question-Answering bot. Your job is to follow the **ARR prompting framework:**

* **A (Analyze):** Carefully read the **retrieved context** and understand the question along with all four options.
* **R (Reason):** Reason step-by-step by matching each option against the context.
* **R (Respond):** Select the correct option **strictly based on the retrieved context**.

**Important Rules:**

* You must **only rely on the retrieved context**. Do not make assumptions or bring in external knowledge.
* In your output:

  * The **Answer** field should contain the **full text of the correct option** .
  * The **Reasoning** should clearly explain why this option is correct and why others are not, based on the context.
  * When generating output **Answer** field should not refer to options using their labels (e.g., "Option A", "Option B", etc.). Instead, directly include the full text of the correct option as the answer.

**Output must strictly follow this JSON structure (no erro

In [85]:

data_file = config['actual_data']
contexts_file = config['rag_retrieved_context']

# Load all data into a list
data_list = []
with open(data_file, 'r', encoding='utf-8') as df:
    for line in df:
        data_list.append(json.loads(line))

# Load contexts into a dictionary
context_dict = {}
with open(contexts_file, 'r', encoding='utf-8') as cf:
    for line in cf:
        entry = json.loads(line)
        question = entry.get('Question')
        retrieved_context = entry.get('retrieved_context')
        if question and retrieved_context:
            context_dict[question] = retrieved_context


def generate_content(data_entry):
    Question=data_entry['Question']
    OptionA=data_entry['A']
    OptionB=data_entry['B']
    OptionC=data_entry['C']
    OptionD=data_entry['D']
    context=data_entry['retrieved_context']
    text='''\n        Question: '''+Question+'''\n            "A":'''+OptionA+'''\n            "B":'''+OptionB+'''\n            "C":'''+OptionC+'''\n            "D":'''+OptionD+'''\n "retrieved_context":'''+context+'''\n   \n    '''
    return text



# ✅ Function to fetch a data point + its retrieved context as a list
def get_data_point_with_context_list(data_entry):

    record = data_entry
    question = record.get('Question')
    retrieved_context = context_dict.get(question)

    record_with_context = record.copy()
    record_with_context['retrieved_context'] = retrieved_context

    # Convert to list of [key, value]


    return record_with_context
# # Example usage:
point_list = get_data_point_with_context_list(data_list[0])


In [ ]:
from huggingface_hub import login
login('')


In [7]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
def extract_nested_braces(s):
    # s = s.replace('"','')
    s += "}"
    stack = []
    start = -1
    result = []
    # print(s)
    for i, char in enumerate(s):
        if char == '{':
            if not stack:
                start = i
            stack.append(char)
        elif char == '}':
            if stack:
                stack.pop()
                if not stack:
                    result.append(s[start:i+1])
    return result[-1] if result else None

def string_to_dict(json_string):
    try:
        # Parse the JSON string into a dictionary
        return json.loads(json_string)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON string: {e}")


def model_output(content, system_prompt_1):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": content}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(response)
    result = string_to_dict(extract_nested_braces(response))

    answer = (
        result.get("Answer") or
        result.get("answer") or
        result.get("correct_answer")
    )
    reasoning = (
        
        result.get("Reasoning") or
        result.get("reasoning")
    )

    return answer, reasoning




In [88]:
content=generate_content(point_list)

In [92]:
output=model_output(content,system_prompt)

```json
{
    "Question": "What legal actions should the authorities consider based on the specific allegations of assault highlighted in the passage, particularly concerning the victim's age and the nature of the crimes?",
    "A": "Authorities should only provide counseling to the victim and not pursue legal actions.",
    "B": "Authorities must take immediate legal action to investigate the allegations as the victim is a minor and the offenses fall under serious criminal categories.",
    "C": "The authorities have a duty to dismiss the allegations immediately due to political involvement.",
    "D": "Legal action can be postponed until the victim is older.",
    "retrieved_context": "When the claim of offence; the court shall make an inquiry, take such evidence as may be necessary (but juvenility is raised before not an affidavit) so as to determine the age of such person, and shall record a finding whether the person is a juvenile or a child or not, stating his age as nearly as ma

In [81]:
output

(None, None)

In [68]:
embedding_model = SentenceTransformer("paraphrase-mpnet-base-v2")

def get_best_option(data, predicted):

    # Extract only the options (A, B, C, D)
    options = {key: data[key] for key in ['A', 'B', 'C', 'D']}

    # Generate embeddings
    predicted_embedding = embedding_model.encode(predicted, convert_to_numpy=True)
    option_embeddings = {key: embedding_model.encode(val, convert_to_numpy=True) for key, val in options.items()}

    # Compute cosine similarities
    similarities = {key: cosine_similarity([predicted_embedding], [emb])[0][0] for key, emb in option_embeddings.items()}

    # Get the best-matching option
    best_option = max(similarities, key=similarities.get)
    return best_option

In [46]:
get_best_option(data_list[100],output[0])

'A'

In [47]:
import os
import json

# Define model name
model_name = config['save_file_main_model']

# Define log directories using os.path.join for better cross-platform compatibility
base_path = "/content"
# base_path = r"/DATA/harsh_2311ai14/Akash_QA/MTP02052025/QA_architecture"
prompt_log = os.path.join(base_path, "prompt", model_name)
result_log = os.path.join(base_path, "result_log", model_name)
accuracy_log = os.path.join(base_path, "accuracy", model_name)

# Ensure directories exist
os.makedirs(prompt_log, exist_ok=True)
os.makedirs(result_log, exist_ok=True)
os.makedirs(accuracy_log, exist_ok=True)


In [ ]:
data_path = config['actual_data']
contexts_file = config['rag_retrieved_context']

# Load all data into a list
data_list = []
with open(data_path, 'r', encoding='utf-8') as df:
    for line in df:
        data_list.append(json.loads(line))

file_name = os.path.basename(data_path)  # Extract file name
print(f"Processing file: {file_name}")

Processing file: data.jsonl


In [ ]:
os.path.join(result_log,  f'result_{model_name}_main_architecture.jsonl')

'/DATA/harsh_2311ai14/Akash_QA/MTP02052025/QA_architecture/result_log/qwen7b/result_qwen7b_main_architecture.jsonl'

In [ ]:

data_path = config['actual_data']
contexts_file = config['rag_retrieved_context']

# Load all data into a list
data_list = []
with open(data_path, 'r', encoding='utf-8') as df:
    for line in df:
        data_list.append(json.loads(line))

file_name = os.path.basename(data_path)  # Extract file name
print(f"Processing file: {file_name}")

data=data_list

# --- ARR routing thresholds ---
ARR_CONF_MARGIN = 0.15  # similarity margin between top-2 options to consider high-confidence
MIN_CONTEXT_LEN = 200   # minimal retrieved_context length for sufficiency proxy

try:
    # Generate system prompt and save it
    system_prompt_1 = system_prompt
    prompt_log_path = os.path.join(prompt_log, f'prompt_{model_name}_main_archi.txt')
    with open(prompt_log_path, "w") as file:
        file.write(system_prompt_1)

    test_data = data # Use the same slicing logic
    predictions = []
    true_labels = []
    result_log_path = os.path.join(result_log,  f'result_{model_name}_main_architecture.json')

    # Load existing output_log if it exists
    if os.path.isfile(result_log_path):
        with open(result_log_path, 'r', encoding="utf-8") as file:
            output_log = [json.loads(line) for line in file]
    else:
        output_log = []

    # Create a set of already processed questions
    processed_questions = set(entry["Question"] for entry in output_log)

    # Process test data
    for i, data1 in enumerate(test_data):
        if data1["Question"] in processed_questions:
            continue  # Skip already processed data points

        try:
            content_retrieved = get_data_point_with_context_list(data1)
            final_content=generate_content(content_retrieved)

            predict, model_reasoning = model_output(content, system_prompt_1)

            # Map free-text answer to option using semantic similarity
            predicted_option = get_best_option(data1, predict)
            actual = get_best_option(data1, data1["Correct Answer"])

            # Compute similarity scores and confidence margin
            options = {k: data1[k] for k in ['A','B','C','D']}
            emb_pred = embedding_model.encode(predict, convert_to_numpy=True)
            sims = {k: cosine_similarity([emb_pred], [embedding_model.encode(v, convert_to_numpy=True)])[0][0] for k,v in options.items()}
            sorted_sims = sorted(sims.values(), reverse=True)
            margin = (sorted_sims[0] - sorted_sims[1]) if len(sorted_sims) >= 2 else 0.0

            # Context sufficiency proxy
            ctx = content_retrieved.get('retrieved_context')
            ctx_len = len(ctx) if isinstance(ctx, str) else 0
            route = 'simple' if (margin >= ARR_CONF_MARGIN and ctx_len >= MIN_CONTEXT_LEN) else 'full'

            predictions.append(predicted_option)
            true_labels.append(actual)

            # Update the log with new data
            temp_dict = {"Question": data1["Question"]}
            temp_dict["Correct Answer"] = actual
            temp_dict["Predicted"] = predicted_option
            temp_dict["main_model_reasoning"] = model_reasoning
            temp_dict["arr_similarity_margin"] = margin
            temp_dict["retrieved_context_len"] = ctx_len
            temp_dict["route"] = route  # 'simple' suggests skipping CDA; 'full' suggests running CDA
            output_log.append(temp_dict)

            print(f"Data {i} saved | route={route} margin={margin:.3f} ctx_len={ctx_len}")

            # Save updated output_log in JSONL format
            with open(result_log_path, 'w', encoding="utf-8") as file:
                for entry in output_log:
                    file.write(json.dumps(entry, ensure_ascii=False) + "\n")

        except Exception as e:
            print(f"An error occurred while processing data: {e}")
            continue  # Skip any errors and proceed to the next item

    # Calculate accuracy
    correct_predictions = [1 if pred == true else 0 for pred, true in zip(predictions, true_labels)]
    accuracy = sum(correct_predictions) / len(correct_predictions) * 100 if true_labels else 0

    # Log accuracy
    accuracy_log_path = os.path.join(accuracy_log, f"{model_name}.txt")
    with open(accuracy_log_path, "a", encoding="utf-8") as file:
        log_entry = f"ModelId: {model_name}\nFilename: {file_name.replace('.jsonl', '')}\nAccuracy: {accuracy:.2f}%\n\n"
        file.write(log_entry)

    print(f"Accuracy for {file_name}: {accuracy:.2f}%")

except Exception as e:
    print(f"An error occurred while processing {file_name}: {e}")


Processing file: data.jsonl
Option B: The law employs specific norms for evaluating witness credibility, especially from minors, ensuring their protection and validation in judicial proceedings.
Based on the retrieved context, particularly the sections discussing the POCSO Act's procedures for handling sexual abuse cases, it is clear that the law takes special care to evaluate and validate the testimonies of minor victims, emphasizing their protection and credibility. Option B directly addresses this aspect, indicating that the law employs specific norms for evaluating witness credibility, especially from minors, ensuring their protection and validation in judicial proceedings.
HII
HII
{'Question': "What legal actions should the authorities consider based on the specific allegations of assault highlighted in the passage, particularly concerning the victim's age and the nature of the crimes?"}
Data 0 saved


KeyboardInterrupt: 

In [54]:
actual

'B'

In [55]:
predictions

[]

In [56]:
predict

'D'